In [ ]:
xml = '''
<?xml version="1.0"?>
<Catalog>
    <Book id="bk101">
        <Author>Garghentini, Davide</Author>
        <Title>XML Developer's Guide</Title>
        <Genre>Computer</Genre>
        <Price>44.95</Price>
        <PublishDate>2000-10-01</PublishDate>
        <Description> An in-depth look at creating applications with XML. </Description>
    </Book>
    <Book id="bk102">
        <Author>Garcia, Debra</Author>
        <Title>Midnight Rain</Title>
        <Genre>Fantasy</Genre>
        <Price>5.95</Price>
        <PublishDate>2000-12-16</PublishDate>
        <Description> A former architect battles corporate zombies, an evil sorceress, and her own childhood to become queen of the world. </Description>
    </Book>
</Catalog>
'''

with open('book.xml', 'w') as f:
    f.write(xml)

In [ ]:
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.3/317.3 MB 3.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.3-py2.py3-none-any.whl size=317840625 sha256=be495020d243ed322d111ade55c8c5fd4cd170c56cf68414e9b3c1c74ec96bc9
  Stored in directory: /root/.cache/pip/wheels/1b/3a/92/28b93e2fbfdbb07509ca4d6f50c5e407f48dce4ddbda69a4ab
Successfully built pyspark


In [ ]:
from pyspark.sql import SparkSession

spark = SparkSession \
    .builder \
    .appName("Test") \
    .config("spark.jars.packages", "com.databricks:spark-xml_2.12:0.13.0") \
    .getOrCreate()

df = spark.read \
    .format('xml') \
    .options(RootTag='Catalog', rowTag='Book') \
    .load('book.xml')

df.printSchema()

root
 |-- Author: string (nullable = true)
 |-- Description: string (nullable = true)
 |-- Genre: string (nullable = true)
 |-- Price: double (nullable = true)
 |-- PublishDate: date (nullable = true)
 |-- Title: string (nullable = true)
 |-- _id: string (nullable = true)



In [ ]:
import pyspark.sql.functions as f

spark.read \
    .format('xml') \
    .options(RootTag='Catalog', rowTag='Book') \
    .load('book.xml') \
    .withColumn('price', f.col("price").cast("double")) \
    .agg(f.max('price')) \
    .show()

+----------+
|max(price)|
+----------+
|     44.95|
+----------+



# Apache Avro

In [ ]:
!pip install avro-python3

  Preparing metadata (setup.py) ... done
  Created wheel for avro-python3: filename=avro_python3-1.10.2-py3-none-any.whl size=43994 sha256=dfc470e71879e1acb29fa3dc3a20e03cbc058b96edc1c02eb6b7fcea85bbb416
  Stored in directory: /root/.cache/pip/wheels/bc/85/62/6cdd81c56f923946b401cecff38055b94c9b766927f7d8ca82
Successfully built avro-python3


In [ ]:
import copy
import json
import avro
from avro.io import DatumWriter, DatumReader
from avro.datafile import DataFileWriter, DataFileReader

In [ ]:
schema = {
    'name': 'avro.example.User',
    'type': 'record',
    'fields': [
        {'name': 'name', 'type': 'string'},
        {'name': 'age', 'type': 'int'}
    ]
}

schema_parsed = avro.schema.Parse(json.dumps(schema))
schema_parsed.__str__()

'{"type": "record", "name": "User", "namespace": "avro.example", "fields": [{"type": "string", "name": "name"}, {"type": "int", "name": "age"}]}'

In [ ]:
with open('users.avro', 'wb') as f:
    writer = DataFileWriter(f, DatumWriter(), schema_parsed)
    writer.append({'name': 'Pierre-Simon Laplace', 'age': 77})
    writer.append({'name': 'John von Neumann', 'age': 53})
    writer.close()


In [ ]:
with open('users.avro', 'rb') as f:
    reader = DataFileReader(f, DatumReader())
    metadata = copy.deepcopy(reader.meta)
    schema_from_file = json.loads(metadata['avro.schema'])
    users = [user for user in reader]
    reader.close()

print(f'Schema that we specified:\n {schema}')
print(f'Schema that we parsed:\n {schema_parsed}')
print(f'Schema from users.avro file:\n {schema_from_file}')
print(f'Users:\n {users}')

AssertionError: -2600952

In [ ]:
schema_new = {
    'name': 'avro.example.User',
    'type': 'record',
    'fields': [
        {'name': 'name', 'type': 'string'},
        {'name': 'age', 'type': 'int'},
        {'name': 'favoriteNumber', 'type': 'int'}
    ]
}

with open('users_new.avro', 'wb') as f:
    writer = DataFileWriter(f, DatumWriter(), avro.schema.Parse(json.dumps(schema_new)))
    writer.append({'name': 'Pierre-Simon Laplace', 'age': 77, 'favoriteNumber': 1337})
    writer.append({'name': 'John von Neumann', 'age': 53, 'favoriteNumber': 228})
    writer.close()

In [ ]:
with open('users_new.avro', 'rb') as f:
    reader = DataFileReader(f, DatumReader(schema_parsed, schema_parsed))
    metadata = copy.deepcopy(reader.meta)
    schema_from_file = json.loads(metadata['avro.schema'])
    users = [user for user in reader]
    reader.close()

print(f'Schema that we specified:\n {schema}')
print(f'Schema that we parsed:\n {schema_parsed}')
print(f'Schema from users.avro file:\n {schema_from_file}')
print(f'Users:\n {users}')

Schema that we specified:
 {'name': 'avro.example.User', 'type': 'record', 'fields': [{'name': 'name', 'type': 'string'}, {'name': 'age', 'type': 'int'}]}
Schema that we parsed:
 {"type": "record", "name": "User", "namespace": "avro.example", "fields": [{"type": "string", "name": "name"}, {"type": "int", "name": "age"}]}
Schema from users.avro file:
 {'type': 'record', 'name': 'User', 'namespace': 'avro.example', 'fields': [{'type': 'string', 'name': 'name'}, {'type': 'int', 'name': 'age'}, {'type': 'int', 'name': 'favoriteNumber'}]}
Users:
 [{'name': 'Pierre-Simon Laplace', 'age': 77}, {'name': 'John von Neumann', 'age': 53}]


In [ ]:
schema_new = {
    'name': 'avro.example.User',
    'type': 'record',
    'fields': [
        {'name': 'name', 'type': 'string'},
        {'name': 'age', 'type': 'string'},
        {'name': 'favoriteNumber', 'type': 'int'}
    ]
}

with open('users_corrupted.avro', 'wb') as f:
    writer = DataFileWriter(f, DatumWriter(), avro.schema.Parse(json.dumps(schema_new)))
    writer.append({'name': 'Pierre-Simon Laplace', 'age': '77', 'favoriteNumber': 1337})
    writer.append({'name': 'John von Neumann', 'age': '53', 'favoriteNumber': 228})
    writer.close()

In [ ]:
with open('users_corrupted.avro', 'rb') as f:
    reader = DataFileReader(f, DatumReader(schema_parsed, schema_parsed))
    metadata = copy.deepcopy(reader.meta)
    schema_from_file = json.loads(metadata['avro.schema'])
    users = [user for user in reader]
    reader.close()

print(f'Schema that we specified:\n {schema}')
print(f'Schema that we parsed:\n {schema_parsed}')
print(f'Schema from users.avro file:\n {schema_from_file}')
print(f'Users:\n {users}')

SchemaResolutionException: Schemas do not match.
Writer's Schema: "string"
Reader's Schema: "int"

In [ ]:
with open('users_corrupted.avro', 'rb') as f:
    reader = DataFileReader(f, DatumReader())
    metadata = copy.deepcopy(reader.meta)
    schema_from_file = json.loads(metadata['avro.schema'])
    users = [user for user in reader]
    reader.close()

print(f'Schema that we specified:\n {schema}')
print(f'Schema that we parsed:\n {schema_parsed}')
print(f'Schema from users.avro file:\n {schema_from_file}')
print(f'Users:\n {users}')

Schema that we specified:
 {'name': 'avro.example.User', 'type': 'record', 'fields': [{'name': 'name', 'type': 'string'}, {'name': 'age', 'type': 'int'}]}
Schema that we parsed:
 {"type": "record", "name": "User", "namespace": "avro.example", "fields": [{"type": "string", "name": "name"}, {"type": "int", "name": "age"}]}
Schema from users.avro file:
 {'type': 'record', 'name': 'User', 'namespace': 'avro.example', 'fields': [{'type': 'string', 'name': 'name'}, {'type': 'string', 'name': 'age'}, {'type': 'int', 'name': 'favoriteNumber'}]}
Users:
 [{'name': 'Pierre-Simon Laplace', 'age': '77', 'favoriteNumber': 1337}, {'name': 'John von Neumann', 'age': '53', 'favoriteNumber': 228}]


---

# Apache Parquet

In [ ]:
!pip install pyarrow

In [ ]:
import pandas as pd
import pyarrow as pa
import pyarrow.parquet as pq

In [ ]:
weather_schema = pa.schema([
    ('city', pa.string()),
    ('measurement_time', pa.timestamp('ms')),
    ('temperature', pa.float32()),
    ('atmospheric_pressure', pa.float32())
])

In [ ]:
from datetime import datetime

In [ ]:
df = pd.DataFrame({
    'city': ['New York', 'London', 'Tokyo'],
    'measurement_time': [
        datetime(2022, 5, 1, 12, 0, 0),
        datetime(2022, 5, 1, 13, 0, 0),
        datetime(2022, 5, 1, 14, 0, 0)],
    'temperature': [20.5, 15.2, 23.1],
    'atmospheric_pressure': [101.5, 99.2, 100.1]
})

table = pa.Table.from_pandas(df, schema=weather_schema)

pq.write_table(table, 'weather_data.parquet')

In [ ]:
# Define the output path for the Parquet dataset
output_path = 'weather_partitioned_data'

# Write the data to a Parquet dataset partitioned by city
pq.write_to_dataset(
    table,
    root_path=output_path,
    partition_cols=['city']
)

---

# Compressing dataset

https://www.kaggle.com/datasets/stefanoleone992/ea-sports-fc-24-complete-player-dataset/

In [ ]:
!unzip archive.zip

Archive:  archive.zip
  inflating: female_coaches.csv      
  inflating: female_players.csv      
  inflating: female_teams.csv        
  inflating: male_coaches.csv        
  inflating: male_players.csv        
  inflating: male_teams.csv          


In [ ]:
import pandas as pd

big_df = pd.read_csv('male_players.csv')
big_df

<ipython-input-17-1d4041304ba6>:3: DtypeWarning: Columns (108) have mixed types. Specify dtype option on import or set low_memory=False.
  big_df = pd.read_csv('male_players.csv')


,player_id,player_url,fifa_version,fifa_update,update_as_of,short_name,long_name,player_positions,overall,potential,...,ldm,cdm,rdm,rwb,lb,lcb,cb,rcb,rb,gk
0,231747,/player/231747/kylian-mbappe/240002,24.0,2.0,2023-09-22,K. Mbappé,Kylian Mbappé Lottin,"ST, LW",91,94,...,63+3,63+3,63+3,68+3,63+3,54+3,54+3,54+3,63+3,18+3
1,239085,/player/239085/erling-haaland/240002,24.0,2.0,2023-09-22,E. Haaland,Erling Braut Haaland,ST,91,94,...,63+3,63+3,63+3,62+3,60+3,62+3,62+3,62+3,60+3,19+3
2,192985,/player/192985/kevin-de-bruyne/240002,24.0,2.0,2023-09-22,K. De Bruyne,Kevin De Bruyne,"CM, CAM",91,91,...,80+3,80+3,80+3,79+3,75+3,70+3,70+3,70+3,75+3,21+3
3,158023,/player/158023/lionel-messi/240002,24.0,2.0,2023-09-22,L. Messi,Lionel Andrés Messi Cuccittini,"CF, CAM",90,90,...,63+3,63+3,63+3,64+3,59+3,49+3,49+3,49+3,59+3,19+3
4,165153,/player/165153/karim-benzema/240002,24.0,2.0,2023-09-22,K. Benzema,Karim Benzema,"CF, ST",90,90,...,64+3,64+3,64+3,64+3,60+3,55+3,55+3,55+3,60+3,18+3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
180016,220806,https://sofifa.com/player/220806/ellis-redman/...,15.0,2.0,2014-09-18,E. Redman,Ellis Redman,CB,41,61,...,38,38,38,37,40,41,41,41,40,10
180017,225509,https://sofifa.com/player/225509/aaron-collins...,15.0,2.0,2014-09-18,A. Collins,Aaron Graham John Collins,ST,41,50,...,30,30,30,31,32,31,31,31,32,14
180018,217591,https://sofifa.com/player/217591/piotr-zemlo/1...,15.0,2.0,2014-09-18,P. Żemło,Piotr Żemło,"LM, LB",40,50,...,46,46,46,48,51-1,53-3,53-3,53-3,51-1,12
180019,222947,https://sofifa.com/player/222947/tom-davies/15...,15.0,2.0,2014-09-18,T. Davies,Thomas Christopher Davies,CB,40,49,...,36,36,36,37,39,40,40,40,39,13


In [ ]:
big_df = big_df[[
    'player_id', 'player_url', 'fifa_version',
    'fifa_update', 'update_as_of', 'short_name',
    'long_name', 'player_positions', 'overall',
    'potential'
]]
big_df

,player_id,player_url,fifa_version,fifa_update,update_as_of,short_name,long_name,player_positions,overall,potential
0,231747,/player/231747/kylian-mbappe/240002,24.0,2.0,2023-09-22,K. Mbappé,Kylian Mbappé Lottin,"ST, LW",91,94
1,239085,/player/239085/erling-haaland/240002,24.0,2.0,2023-09-22,E. Haaland,Erling Braut Haaland,ST,91,94
2,192985,/player/192985/kevin-de-bruyne/240002,24.0,2.0,2023-09-22,K. De Bruyne,Kevin De Bruyne,"CM, CAM",91,91
3,158023,/player/158023/lionel-messi/240002,24.0,2.0,2023-09-22,L. Messi,Lionel Andrés Messi Cuccittini,"CF, CAM",90,90
4,165153,/player/165153/karim-benzema/240002,24.0,2.0,2023-09-22,K. Benzema,Karim Benzema,"CF, ST",90,90
...,...,...,...,...,...,...,...,...,...,...
180016,220806,https://sofifa.com/player/220806/ellis-redman/...,15.0,2.0,2014-09-18,E. Redman,Ellis Redman,CB,41,61
180017,225509,https://sofifa.com/player/225509/aaron-collins...,15.0,2.0,2014-09-18,A. Collins,Aaron Graham John Collins,ST,41,50
180018,217591,https://sofifa.com/player/217591/piotr-zemlo/1...,15.0,2.0,2014-09-18,P. Żemło,Piotr Żemło,"LM, LB",40,50
180019,222947,https://sofifa.com/player/222947/tom-davies/15...,15.0,2.0,2014-09-18,T. Davies,Thomas Christopher Davies,CB,40,49


In [ ]:
big_df.to_csv('big_df.csv', index=False)

In [ ]:
!wc -c big_df.csv

22145807 big_df.csv


In [ ]:
big_df.to_json('big_df.json')

In [ ]:
!wc -c big_df.json

40418168 big_df.json


In [ ]:
big_df.columns

Index(['player_id', 'player_url', 'fifa_version', 'fifa_update',
       'update_as_of', 'short_name', 'long_name', 'player_positions',
       'overall', 'potential'],
      dtype='object')

In [ ]:
schema_new = {
    'name': 'avro.example.Player',
    'type': 'record',
    'fields': [
        {'name': 'player_id', 'type': 'int'},
        {'name': 'player_url', 'type': 'string'},
        {'name': 'fifa_version', 'type': 'float'},
        {'name': 'fifa_update', 'type': 'float'},
        {'name': 'update_as_of', 'type': 'string'},
        {'name': 'short_name', 'type': 'string'},
        {'name': 'long_name', 'type': 'string'},
        {'name': 'player_positions', 'type': 'string'},
        {'name': 'overall', 'type': 'int'},
        {'name': 'potential', 'type': 'int'},
    ]
}

In [ ]:
with open('players.avro', 'wb') as f:
    writer = DataFileWriter(f, DatumWriter(), avro.schema.Parse(json.dumps(schema_new)))
    for index, row in big_df.iterrows():
        writer.append({i: row[i] for i in big_df.columns})
    writer.close()



In [ ]:
!wc -c players.avro

20665547 players.avro


In [ ]:
big_df.to_parquet('players_raw.parquet', compression=None)

In [ ]:
!wc -c players_raw.parquet

14191645 players_raw.parquet


In [ ]:
big_df.to_parquet('players.parquet')

In [ ]:
!wc -c players.parquet

7388563 players.parquet
